In [2]:
import openfermion
import numpy as np
import copy as cp
import scipy.sparse.linalg as ssl # Muqing: for Pauli Excitation

from openfermion import *
import operator_pools

This is a modified operator_pools.py by Muqing Zheng


In [3]:
n_orb = 4
n_a = n_orb // 2
n_b = n_orb // 2

pool = operator_pools.singlet_GSD()
pool.init(n_orb, n_occ_a=n_a, n_occ_b=n_b, n_vir_a=n_orb-n_a, n_vir_b=n_orb-n_b)

 Form singlet GSD operators
Ansatz 0 0
Ansatz 0 1
  Single: 0 1
Ansatz 0 2
  Single: 0 2
Ansatz 0 3
  Single: 0 3
Ansatz 1 1
Ansatz 1 2
  Single: 1 2
Ansatz 1 3
  Single: 1 3
Ansatz 2 2
Ansatz 2 3
  Single: 2 3
Ansatz 3 3
  Singlet2t: 0 1 0 0
-0.5 [1^ 0^ 2 1] +
0.5 [1^ 0^ 3 0] +
0.5 [2^ 1^ 1 0] +
-0.5 [3^ 0^ 1 0]

  Singlet2t: 0 2 0 0
-0.5 [1^ 0^ 4 1] +
0.5 [1^ 0^ 5 0] +
0.5 [4^ 1^ 1 0] +
-0.5 [5^ 0^ 1 0]

  Singlet2t: 0 3 0 0
-0.5 [1^ 0^ 6 1] +
0.5 [1^ 0^ 7 0] +
0.5 [6^ 1^ 1 0] +
-0.5 [7^ 0^ 1 0]

  Singlet1t: 1 1 0 0
0.7071067811865475 [1^ 0^ 3 2] +
-0.7071067811865475 [3^ 2^ 1 0]

  Singlet2t: 1 2 0 0
-0.5 [1^ 0^ 4 3] +
0.5 [1^ 0^ 5 2] +
0.5 [4^ 3^ 1 0] +
-0.5 [5^ 2^ 1 0]

  Singlet2t: 1 3 0 0
-0.5 [1^ 0^ 6 3] +
0.5 [1^ 0^ 7 2] +
0.5 [6^ 3^ 1 0] +
-0.5 [7^ 2^ 1 0]

  Singlet1t: 2 2 0 0
0.7071067811865475 [1^ 0^ 5 4] +
-0.7071067811865475 [5^ 4^ 1 0]

  Singlet2t: 2 3 0 0
-0.5 [1^ 0^ 6 5] +
0.5 [1^ 0^ 7 4] +
0.5 [6^ 5^ 1 0] +
-0.5 [7^ 4^ 1 0]

  Singlet1t: 3 3 0 0
0.7071067811865475 

In [4]:
pool.fermi_ops[0]

0.5 [0^ 2] +
0.5 [1^ 3] +
-0.5 [2^ 0] +
-0.5 [3^ 1]

In [5]:
single_count = 0
double_count = 0
terms_count = {}
for p in pool.fermi_ops:
    # print(len(p.terms), len(list(p.terms.keys())[0]))
    if len(list(p.terms.keys())[0]) == 4:
        double_count += 1
        if len(p.terms) in terms_count:
            terms_count[len(p.terms)] += 1
        else:
            terms_count[len(p.terms)] = 1
    else:
        single_count += 1
print(f"Number of single excitations {single_count}")
print(f"Number of double excitations {double_count}")
terms_count

Number of single excitations 6
Number of double excitations 60


{4: 24, 2: 6, 12: 15, 8: 15}

In [4]:
n_orb = 4

singles = []
doubles_singlet1 = []
doubles_singlet2 = []
doubles_singlet4 = []
double_triplet = []
total_count = 0


for p in range(n_orb):
    pa = 2*p
    pb = 2*p+1
    for q in range(p+1, n_orb):
        qa = 2*q
        qb = 2*q+1

        total_count += 1
        singles.append((pa, qa, pb, qb))

rs = -1
for r in range(n_orb):
    for s in range(r, n_orb):
        rs += 1
        pq = -1
        for p in range(n_orb):
            for q in range(p, n_orb):
                pq += 1

                if rs > pq:
                    continue

                if p == r and q == s:
                    continue
                
                ## 
                if p != q:
                    if r ==s:
                        doubles_singlet2.append((p, q, r, s))
                        total_count += 1
                        continue
                    if r != s and q != r:
                        double_triplet.append((p, q, r, s))
                        doubles_singlet4.append((p, q, r, s))
                        total_count += 1
                        continue
                    if p == r and q != s and r!=s:
                        double_triplet.append((p, q, r, s))
                        doubles_singlet4.append((p, q, r, s))
                        total_count += 1
                        continue
                    if p == s and q != r and r!=s:
                        double_triplet.append((p, q, r, s))
                        doubles_singlet4.append((p, q, r, s))
                        total_count += 1
                        continue
                    if q == s and p != r:
                        double_triplet.append((p, q, r, s))
                        doubles_singlet4.append((p, q, r, s))
                        total_count += 1
                        continue
                    if q == r and p != s:
                        double_triplet.append((p, q, r, s))
                        doubles_singlet4.append((p, q, r, s))
                        total_count += 1
                        continue

                ##
                if p == q:
                    if q != r and r != s:
                        doubles_singlet2.append((p, q, r, s))
                        total_count += 1
                        continue
                    if q == r and r != s:
                        doubles_singlet2.append((p, q, r, s))
                        total_count += 1
                        continue
                    if q != r and r == s:
                        doubles_singlet1.append((p, q, r, s))
                        total_count += 1
                        continue
                

print(f"Total count {total_count}")
print(f"  Single excitations                      {len(singles)}")
print(f"  Double excitations                      {len(doubles_singlet1) + len(doubles_singlet2) + len(doubles_singlet4) + len(double_triplet)}")
print(f"    Double excitations singlet one term   {len(doubles_singlet1)}")
print(f"    Double excitations singlet two terms  {len(doubles_singlet2)}")
print(f"    Double excitations singlet four terms {len(doubles_singlet4)}")
print(f"    Double excitations triplet            {len(double_triplet)}")

Total count 51
  Single excitations                      6
  Double excitations                      60
    Double excitations singlet one term   6
    Double excitations singlet two terms  24
    Double excitations singlet four terms 15
    Double excitations triplet            15


In [16]:
def count_singlet_gsd_excitations(n_orbs):
    # Singles: all p,q pairs
    n_singles = n_orbs * (n_orbs - 1)  # For n=4: 4*3 = 12
    
    # Doubles breakdown:
    # Term 1: One operator terms (p==q cases with q!=r && r==s)
    term1 = n_orbs * (n_orbs - 1) // 2  # For n=4: 4*3/2 = 6
    
    # Term 2: Two operator terms
    term2 = n_orbs * (n_orbs - 1) * (n_orbs - 2)  # For n=4: 4*3*2 = 24
    
    # Term 4: Four operator terms (singlet)
    term4 = n_orbs * (n_orbs - 1) * (n_orbs - 2) // 2  # For n=4: 4*3*2/2 = 15
    
    # Term 6: Six operator terms (triplet)
    term6 = n_orbs * (n_orbs - 1) * (n_orbs - 2) // 2  # For n=4: 4*3*2/2 = 15
    
    # Total number of operators in doubles
    n_doubles = term1 + 2*term2 + 4*term4 + 6*term6
    
    print(f"For n_orbs = {n_orbs}:")
    print(f"Singles: {n_singles}")
    print("Doubles breakdown:")
    print(f"  - One operator terms:   {term1}")
    print(f"  - Two operator terms:   {term2}")
    print(f"  - Four operator terms:  {term4}")
    print(f"  - Six operator terms:   {term6}")
    print(f"Total doubles operators:  {n_doubles}")
    print(f"Total operators:          {n_singles + n_doubles}")
    
    return n_singles + n_doubles

# Test with n_orbs = 4
count_singlet_gsd_excitations(4)


For n_orbs = 4:
Singles: 12
Doubles breakdown:
  - One operator terms:   6
  - Two operator terms:   24
  - Four operator terms:  12
  - Six operator terms:   12
Total doubles operators:  174
Total operators:          186


186